In [3]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score

pd.set_option("display.max_columns", 30)

In [4]:
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")
sample_submission = pd.read_csv("sample_submission.csv")

In [14]:
train_df

,age,height(cm),weight(kg),waist(cm),eyesight(left),eyesight(right),hearing(left),hearing(right),systolic,relaxation,fasting blood sugar,Cholesterol,triglyceride,HDL,LDL,hemoglobin,Urine protein,serum creatinine,AST,ALT,Gtp,dental caries,smoking
0,60.0,150.0,55.0,76.0,0.7,0.7,1.0,1.0,131.0,86.0,88.0,213.0,63.0,60.0,141.0,13.2,1.0,0.7,23.0,12.0,14.0,1.0,0.0
1,40.0,170.0,60.0,73.7,1.0,0.5,1.0,1.0,100.0,60.0,84.0,160.0,78.0,53.0,92.0,15.7,1.0,1.1,19.0,12.0,18.0,0.0,0.0
2,40.0,155.0,50.0,80.0,1.5,1.2,1.0,1.0,114.0,74.0,85.0,155.0,68.0,44.0,97.0,12.9,1.0,0.6,19.0,9.0,9.0,0.0,0.0
3,50.0,170.0,75.0,88.0,1.2,1.2,1.0,2.0,113.0,69.0,119.0,163.0,109.0,45.0,96.0,15.4,1.0,1.0,28.0,21.0,40.0,0.0,1.0
4,45.0,155.0,60.0,76.0,1.0,0.8,1.0,1.0,134.0,78.0,96.0,184.0,56.0,66.0,107.0,12.1,1.0,0.8,15.0,12.0,11.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14995,40.0,180.0,70.0,78.0,1.0,1.2,1.0,1.0,136.0,86.0,99.0,191.0,111.0,55.0,113.0,15.3,1.0,0.7,17.0,14.0,29.0,0.0,0.0
14996,40.0,175.0,70.0,82.0,1.0,1.0,1.0,1.0,118.0,74.0,104.0,197.0,92.0,58.0,120.0,15.5,1.0,1.0,21.0,21.0,22.0,0.0,1.0
14997,55.0,170.0,60.0,79.0,1.0,1.0,1.0,1.0,102.0,64.0,93.0,204.0,107.0,67.0,116.0,14.3,1.0,1.0,37.0,30.0,27.0,0.0,0.0
14998,50.0,155.0,55.0,80.0,0.7,0.9,1.0,1.0,130.0,80.0,89.0,246.0,176.0,46.0,164.0,13.5,1.0,0.8,25.0,21.0,28.0,0.0,0.0


In [15]:
test_df

,age,height(cm),weight(kg),waist(cm),eyesight(left),eyesight(right),hearing(left),hearing(right),systolic,relaxation,fasting blood sugar,Cholesterol,triglyceride,HDL,LDL,hemoglobin,Urine protein,serum creatinine,AST,ALT,Gtp,dental caries
0,60.0,170.0,60.0,80.0,0.7,0.7,1.0,1.0,124.0,83.0,86.0,184.0,53.0,72.0,102.0,14.9,1.0,0.8,26.0,22.0,26.0,0.0
1,65.0,170.0,70.0,91.0,0.6,0.7,1.0,1.0,130.0,90.0,92.0,178.0,68.0,45.0,120.0,15.2,1.0,1.0,29.0,23.0,23.0,0.0
2,35.0,175.0,75.0,86.4,1.2,1.2,1.0,1.0,124.0,80.0,94.0,189.0,67.0,55.0,121.0,15.1,2.0,0.9,15.0,14.0,18.0,0.0
3,30.0,170.0,65.0,79.0,1.0,0.8,1.0,1.0,114.0,70.0,79.0,182.0,68.0,48.0,120.0,16.5,1.0,0.9,36.0,29.0,25.0,0.0
4,25.0,175.0,60.0,72.0,1.2,0.7,1.0,1.0,106.0,72.0,112.0,212.0,107.0,93.0,98.0,14.8,1.0,1.0,17.0,14.0,52.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,20.0,175.0,70.0,82.1,1.5,1.5,1.0,1.0,130.0,84.0,93.0,170.0,68.0,64.0,92.0,16.5,1.0,0.9,26.0,15.0,18.0,0.0
9996,40.0,165.0,55.0,71.0,1.2,0.8,1.0,1.0,111.0,71.0,92.0,179.0,71.0,68.0,97.0,14.7,1.0,1.0,18.0,13.0,12.0,0.0
9997,40.0,155.0,60.0,84.0,0.1,1.0,1.0,1.0,108.0,65.0,91.0,194.0,70.0,67.0,113.0,12.2,1.0,0.7,17.0,14.0,12.0,0.0
9998,35.0,175.0,85.0,91.0,1.2,1.2,1.0,1.0,131.0,82.0,101.0,187.0,122.0,46.0,116.0,15.1,1.0,0.9,15.0,19.0,39.0,0.0


In [5]:
train_df = train_df.drop(columns=["id"])
test_df_id = test_df["id"]
test_df = test_df.drop(columns=["id"])

In [6]:
X = train_df.drop(columns=["smoking"])
y = train_df["smoking"]
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

In [7]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
test_df_scaled = scaler.transform(test_df)

In [8]:
model = lgb.LGBMClassifier(
    objective="binary",
    metric="auc",
    learning_rate=0.05,
    num_leaves=63,
    max_depth=8,
    n_estimators=200,
    subsample=0.8,
    colsample_bytree=0.8,
    reg_lambda=0.1,
    random_state=42,
    n_jobs=-1
)


In [9]:
model.fit(
    X_train_scaled, y_train,
    eval_set=[(X_val_scaled, y_val)],
    eval_metric="auc",
    callbacks=[
        lgb.early_stopping(stopping_rounds=50),
        lgb.log_evaluation(50)
    ]
)

/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


[LightGBM] [Info] Number of positive: 4361, number of negative: 7639
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.002762 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1456
[LightGBM] [Info] Number of data points in the train set: 12000, number of used features: 22
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.363417 -> initscore=-0.560565
[LightGBM] [Info] Start training from score -0.560565
Training until validation scores don't improve for 50 rounds
[50]	valid_0's auc: 0.875363
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with

LGBMClassifier(colsample_bytree=0.8, learning_rate=0.05, max_depth=8,
               metric='auc', n_estimators=200, n_jobs=-1, num_leaves=63,
               objective='binary', random_state=42, reg_alpha=0.1,
               reg_lambda=0.1, subsample=0.8)

In [10]:
if hasattr(model, "booster_"):
    print("Модель обучена успешно!")
    best_iter = model.best_iteration_
else:
    print("Ошибка: модель не обучилась")

Модель обучена успешно!


In [11]:
y_pred_test = model.predict_proba(test_df_scaled, num_iteration=best_iter)[:, 1]

/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


In [12]:
submission = pd.DataFrame({
    "id": test_df_id,
    "smoking": y_pred_test
})
submission.to_csv("submission.csv", index=False)

In [13]:
print("Примеры предсказаний:")
print(submission.head())

Примеры предсказаний:
      id   smoking
0  15000  0.304205
1  15001  0.227494
2  15002  0.286349
3  15003  0.317162
4  15004  0.898978
